# 1. Data preprocessing

In [ ]:
import imageio
import random
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Activation, Input, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, concatenate, Concatenate, UpSampling2D, AveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras


import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
images_list = []
masks_list = []
def store_images(data):
    image = glob.glob('/kaggle/input/lyft-udacity-challenge/'+data+'/'+data+'/CameraRGB/*.png')
    mask = glob.glob('/kaggle/input/lyft-udacity-challenge/'+data+'/'+data+'/CameraSeg/*.png')
    images_list.extend(image)
    masks_list.extend(mask)   
for data in ['dataA', 'dataB', 'dataC', 'dataD', 'dataE']:
    store_images(data)

dataset = tf.data.Dataset.from_tensor_slices((images_list, masks_list))
dataset_length = sum(1 for _ in dataset)

print("Length of dataset:", dataset_length)
# for image, mask in dataset.take(1):
#     print("Image File:", image.numpy().decode(), "Mask File:", mask.numpy().decode())

In [ ]:
def show_images(data, num):
    # load images
    images = glob.glob('/kaggle/input/lyft-udacity-challenge/'+data+'/'+data+'/CameraRGB/*.png')[:num]
    masks = glob.glob('/kaggle/input/lyft-udacity-challenge/'+data+'/'+data+'/CameraSeg/*.png')[:num]

    for i in range(len(images)):
        images[i] = cv2.imread(images[i])
        masks[i] = cv2.imread(masks[i], cv2.IMREAD_GRAYSCALE)

    fig, axes = plt.subplots(num, 2)

    for i in range(len(images)):
        axes[i][0].imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
        axes[i][1].imshow(masks[i])
        axes[i][0].axis('off')
        axes[i][1].axis('off')
    plt.show()

show_images('dataB', 2)


In [ ]:
def load_datas(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return image, mask

def preprocess(image, mask, height=256, width=256):
    input_image = tf.image.resize(image, (height, width), method='nearest')
    input_mask = tf.image.resize(mask, (height, width), method='nearest')

    return input_image, input_mask

dataset_load = dataset.map(load_datas)
dataset_processed = dataset_load.map(preprocess)

In [ ]:
EPOCHS = 15
BATCH_SIZE = 32

In [ ]:
def split_dataset(data, train_split=0.7, val_split=0.15, test_split=0.15, shuffle=True, shuffle_size=1000, batch_size=BATCH_SIZE):
    """
    Splits the dataset into training, validation, and test sets.
    Returns: (train_dataset, val_dataset, test_dataset).
    """

    assert train_split + val_split + test_split == 1, "Split ratios must sum to 1"

    # Determine the size of the dataset
    dataset_size = len(list(data))

    # Calculate split sizes
    train_size = int(train_split * dataset_size)
    val_size = int(val_split * dataset_size)

    # Shuffle the dataset
    if shuffle:
        data = data.shuffle(shuffle_size, reshuffle_each_iteration=False)

    # Split the dataset
    train_dataset = data.take(train_size)
    val_dataset = data.skip(train_size).take(val_size)
    test_dataset = data.skip(train_size + val_size)

    # Batch the datasets
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)
    test_dataset = test_dataset.batch(batch_size)

    return train_dataset, val_dataset, test_dataset

train_dataset, val_dataset, test_dataset = split_dataset(dataset_processed)

#  2. Model Architectures
In this project, 3 model architectures are built, trained and compared, to see each performance on semantic segmentation tasks for autonomous driving.

### 1. Fully Convolutional Network (FCN)
FCN is a deep learning architecture for semantic segmentation tasks, retaining spatial information with convolutional layers instead of fully connected layers.

### 2. U-Net
U-Net is a CNN architecture initally used for biomedical image segmentation, featuring a contracting and expansive path with skip connections for precise localization.

### 3. DeepLabV3
DeepLabV3, developed by Google Research, is a cutting-edge architecture for semantic image segmentation tasks.


* ###  Model architecture for FCN

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Conv2DTranspose, concatenate, Activation

def conv_block_3(inputs=None, n_filters=32, dropout_prob=0):
    layer = Conv2D(n_filters, 3, padding='same', kernel_initializer='he_normal')(inputs)
    layer = BatchNormalization(axis=3)(layer)
    layer = Activation("relu")(layer)
    layer = Conv2D(n_filters, 3, padding='same', kernel_initializer='he_normal')(layer)
    layer = BatchNormalization(axis=3)(layer)
    layer = Activation("relu")(layer)

    if dropout_prob > 0:
        layer = tf.keras.layers.Dropout(dropout_prob)(layer)

    return layer

def decoder_block(input_tensor, concat_tensor, n_filters):
    # Upsample the input
    decoder = Conv2DTranspose(n_filters, (3, 3), strides=(1, 1), padding='same')(input_tensor)
    # Concatenate with the corresponding layer from the encoder
    decoder = concatenate([decoder, concat_tensor], axis=-1)
    # Additional convolutional layers
    decoder = Conv2D(n_filters, (3, 3), padding='same')(decoder)
    decoder = BatchNormalization()(decoder)
    decoder = Activation('relu')(decoder)

    return decoder

def fcn_model(input_size=(256, 256, 3), n_filters=32, n_classes=13):
    inputs = Input(input_size)

    # Encoder
    cblock1 = conv_block_3(inputs, n_filters) # (None, 256, 256, 32)
    cblock2 = conv_block_3(cblock1, n_filters * 2) # (None, 256, 256, 64)
    cblock3 = conv_block_3(cblock2, n_filters * 4) # (None, 256, 256, 128)

    # Decoder
    dblock1 = decoder_block(cblock3, cblock2, n_filters * 2) # (None, 256, 256, 64)
    dblock2 = decoder_block(dblock1, cblock1, n_filters) # (None, 256, 256, 32)

    # Output layer: (None, 256, 256, 13)
    layer = Conv2D(n_classes, 3, padding='same', kernel_initializer='he_normal')(cblock3)
    layer = BatchNormalization(axis=3)(layer)

    model = tf.keras.Model(inputs=inputs, outputs=layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model

* ###  Model architecture for U-Net

In [ ]:
def conv_layer(inputs, n_filters, kernel_size=3):
    """
    Apply convolution, batch normalization, and ReLU activation.

    Args:
    - inputs: Input tensor.
    - n_filters: Number of filters for the convolutional layer.
    - kernel_size: Size of the convolution kernel.

    Returns:
    - Output tensor after the operations.
    """
    layer = tfl.Conv2D(n_filters, kernel_size=kernel_size, padding='same', kernel_initializer='he_normal')(inputs)
    layer = tfl.BatchNormalization(axis=3)(layer)
    layer = tfl.ReLU()(layer)
    return layer

In [ ]:
def conv_block(inputs, n_filters, dropout_prob=0, max_pooling=True):
    """
    Perform two convolutions with optional dropout and max pooling.

    Args:
    - inputs: Input tensor.
    - n_filters: Number of filters for the convolutional layers.
    - dropout_prob: Dropout rate.
    - max_pooling: Boolean, whether to include a max pooling layer.

    Returns:
    - next_layer: Output tensor for the next layer.
    - skip_connection: Output tensor for the skip connection.
    """
    # Two convolutional layers
    layer = conv_layer(inputs, n_filters)
    layer = conv_layer(layer, n_filters)

    # Optional dropout
    if dropout_prob > 0:
        layer = tfl.Dropout(dropout_prob)(layer)
    # Optional max pooling
    if max_pooling:
        next_layer = tfl.MaxPooling2D(pool_size=(2, 2))(layer)
    else:
        next_layer = layer

    skip_connection = layer

    return next_layer, skip_connection

In [ ]:
def upsampling_block(inputs, skip_connection_inputs, n_filters):
    """
    Upsample the input and merge with the skip connection.

    Args:
    - inputs: Input tensor from the previous layer.
    - skip_connection_inputs: Input tensor from the corresponding contraction block (for the skip connection).
    - n_filters: Number of filters for the convolutional layers.

    Returns:
    - layer: Output tensor after upsampling and convolution.
    """
    # Upsampling
    up = tfl.Conv2DTranspose(n_filters, kernel_size=3, strides=(2, 2), padding='same')(inputs)
    # Merging with skip connection
    merge = tfl.concatenate([up, skip_connection_inputs], axis=3)
    # Two convolutional layers
    layer = conv_layer(merge, n_filters)
    layer = conv_layer(layer, n_filters)

    return layer

In [ ]:
# U-Net model
def unet_model(input_size=(256, 256, 3), n_filters=32, n_classes=13):
    """
    Define the U-Net model architecture.

    Args:
    - input_size: Shape of the input images.
    - n_filters: Number of filters for the convolutional layers in the first block. Gets doubled in each subsequent block.
    - n_classes: Number of output classes.

    Returns:
    - model: Compiled U-Net model.
    """
    inputs = tfl.Input(input_size)

    # Encoding path
    cblock1 = conv_block(inputs, n_filters)
    cblock2 = conv_block(cblock1[0], n_filters * 2)
    cblock3 = conv_block(cblock2[0], n_filters * 4)
    cblock4 = conv_block(cblock3[0], n_filters * 8, dropout_prob=0.3)
    cblock5 = conv_block(cblock4[0], n_filters * 16, dropout_prob=0.3, max_pooling=False)

    # Decoding path
    ublock6 = upsampling_block(cblock5[0], cblock4[1], n_filters * 8)
    ublock7 = upsampling_block(ublock6, cblock3[1], n_filters * 4)
    ublock8 = upsampling_block(ublock7, cblock2[1], n_filters * 2)
    ublock9 = upsampling_block(ublock8, cblock1[1], n_filters)

    # Output layer
    output_layer = conv_layer(ublock9, n_filters)
    output_layer = tfl.Conv2D(n_classes, kernel_size=1, padding='same')(output_layer)

    model = tf.keras.Model(inputs=inputs, outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

    return model

* ###  Model architecture for DeepLabV3

In [ ]:
def aspp(input):
    shape = input.shape
    y_pool = AveragePooling2D(pool_size = (shape[1], shape[2]))(input)
    y_pool = Conv2D(filters=256, padding='same', use_bias=False, kernel_size=1)(y_pool)
    y_pool = BatchNormalization()(y_pool)
    y_pool = Activation("relu")(y_pool)
    y_pool = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y_pool)

    out_1 = Conv2D(filters=256, padding='same', dilation_rate = 1, use_bias=False, kernel_size=1)(input)
    out_1 = BatchNormalization()(out_1)
    out_1 = Activation("relu")(out_1)

    out_6 = Conv2D(filters=256, padding='same', dilation_rate = 6, use_bias=False, kernel_size=1)(input)
    out_6 = BatchNormalization()(out_6)
    out_6 = Activation("relu")(out_6)

    out_12 = Conv2D(filters=256, padding='same', dilation_rate = 12, use_bias=False, kernel_size=1)(input)
    out_12 = BatchNormalization()(out_12)
    out_12 = Activation("relu")(out_12)

    out_18 = Conv2D(filters=256, padding='same', dilation_rate = 18, use_bias=False, kernel_size=1)(input)
    out_18 = BatchNormalization()(out_18)
    out_18 = Activation("relu")(out_18)

    y = Concatenate()([y_pool, out_1, out_6, out_12, out_18])
    y = Conv2D(filters=256, padding='same', dilation_rate = 1, use_bias=False, kernel_size=1)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y

def DeeplabV3(image_size=(256, 256, 3), num_classes=13):

    inputs = Input(image_size)

    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=inputs
    )

    # pretrained Resnet50 output
#     x1 = resnet50.get_layer("conv4_block6_out").output
    x1 = resnet50.get_layer("conv4_block6_2_relu").output
    x1 = aspp(x1)
    x1 = UpSampling2D((4, 4), interpolation="bilinear")(x1)

    # low level features
#     x2 = resnet50.get_layer("conv2_block2_out").output
    x2 = resnet50.get_layer("conv2_block3_2_relu").output
    x2 = Conv2D(filters=48, padding='same', use_bias=False, kernel_size=1)(x2)
    x2 = BatchNormalization()(x2)
    x2 = Activation("relu")(x2)

    x = Concatenate()([x1, x2])

    x = Conv2D(filters=256, padding='same', activation='relu', use_bias=False, kernel_size=3)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters=256, padding='same', activation='relu', use_bias=False, kernel_size=3)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D((4, 4), interpolation="bilinear")(x)

    # output
    x = Conv2D(num_classes, kernel_size=(1, 1), name="output_layer", padding="same")(x)
    x = Activation('sigmoid')(x)

    model = Model(inputs=inputs, outputs=x)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    return model


# 3. Model Training

### Hyperparameter toning and model configerations. 

In [ ]:
img_height = 256
img_width = 256
num_channels = 3
filters = 32
n_classes = 13

### Function to plot training history

In [ ]:
def plot_training_history(history):
    acc = [0.] + history.history.get('accuracy', [])
    val_acc = [0.] + history.history.get('val_accuracy', [])
    loss = history.history.get('loss', [])
    val_loss = history.history.get('val_loss', [])

    # Plotting
    plt.figure(figsize=(8, 8))

    # Accuracy subplot
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.ylim([0, 1])

    # Loss subplot
    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylim([0, 1])

    plt.tight_layout()
    plt.show()

* ### Training FCN

In [ ]:
import time

# FCN model

fcn_model1 = fcn_model()
fcn_model1.summary()

In [ ]:
start_time = time.time()

# reduce_lr = ReduceLROnPlateau(monitor= "loss", factor=0.1,
#                               patience= 1, min_lr= 1e-6)
# early_stop = EarlyStopping(patience= 2)

reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=10, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

fcn_result1 = fcn_model1.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=EPOCHS,
                    callbacks=[reduce_lr,early_stop],
                    batch_size=BATCH_SIZE)
end_time = time.time()
training_time = end_time - start_time

In [ ]:
print("----------------------------------------------------------------")
print("Training time for FCN:", training_time, "seconds")

In [ ]:
plot_training_history(fcn_result1)

* ###  Training U-Net

In [ ]:
# U-net model
unet_model1 = unet_model()
unet_model1.summary()

In [ ]:
start_time = time.time()

unet_model1 = unet_model()
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=10, min_lr=1e-6)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

unet_result1 = unet_model1.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=EPOCHS,
                    callbacks=[reduce_lr,early_stop],
                    batch_size=BATCH_SIZE)
end_time = time.time()

In [ ]:
training_time = end_time - start_time
print("----------------------------------------------------------------")
print("Training time for U-Net:", training_time, "seconds")

In [ ]:
plot_training_history(unet_result1)

* ###  Training DeepLabV3

In [ ]:
# Deeplab model
deeplab_model1 = DeeplabV3()
deeplab_model1.summary()

In [ ]:
start_time = time.time()

early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy',factor=1e-1, patience=10, verbose=1, min_lr = 1e-6)

deeplab_result1 = deeplab_model1.fit(train_dataset, 
                    validation_data = val_dataset, 
                    epochs = EPOCHS, 
                    callbacks=[early_stop, reduce_lr], 
                    batch_size = BATCH_SIZE)
end_time = time.time()

In [ ]:
training_time = end_time - start_time
print("----------------------------------------------------------------")
print("Training time for Deeplab:", training_time, "seconds")

In [ ]:
plot_training_history(deeplab_result1)

# 4. Evaluation

* ### Accuracy

In [ ]:
def model_accuracy(model):
    train_loss, train_accuracy = model.evaluate(train_dataset, batch_size = BATCH_SIZE)
    validation_loss, validation_accuracy = model.evaluate(val_dataset, batch_size = BATCH_SIZE)
    test_loss, test_accuracy = model.evaluate(test_dataset, batch_size = BATCH_SIZE)
    print(f'Model Accuracy on the Training Dataset: {round(train_accuracy * 100, 2)}%')
    print(f'Model Accuracy on the Validation Dataset: {round(validation_accuracy * 100, 2)}%')
    print(f'Model Accuracy on the Test Dataset: {round(test_accuracy * 100, 2)}%')

In [ ]:
model_accuracy(fcn_model1)

In [ ]:
model_accuracy(unet_model1)

In [ ]:
model_accuracy(deeplab_model1)

* ### Intersection-over-Union (IoU)

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd  
import numpy as np

def evaluate_metrics(dataset, model, n_classes=13):
    # create mask
    true_masks, predicted_masks = [], []
    for images, masks in dataset:
        pred_masks = model.predict(images)
        pred_masks = tf.expand_dims(tf.argmax(pred_masks, axis=-1), axis=-1)
        true_masks.extend(masks)
        predicted_masks.extend(pred_masks)
    true_masks = np.array(true_masks)
    predicted_masks = np.array(predicted_masks)
    
    # initialize
    class_wise_metrics = {
        "TP": np.zeros(n_classes),  # True Positives
        "FP": np.zeros(n_classes),  # False Positives
        "FN": np.zeros(n_classes)   # False Negatives
    }

    # calculate metrics per class
    for c_id in range(n_classes):
        true_positive = (predicted_masks == c_id) & (true_masks == c_id)
        false_positive = (predicted_masks == c_id) & (true_masks != c_id)
        false_negative = (predicted_masks != c_id) & (true_masks == c_id)

        class_wise_metrics["TP"][c_id] += np.sum(true_positive)
        class_wise_metrics["FP"][c_id] += np.sum(false_positive)
        class_wise_metrics["FN"][c_id] += np.sum(false_negative)
        
    # class-wise metrics
    recall = np.round(class_wise_metrics["TP"] / (class_wise_metrics["TP"] + class_wise_metrics["FN"] + 1e-6), 2)
    precision = np.round(class_wise_metrics["TP"] / (class_wise_metrics["TP"] + class_wise_metrics["FP"] + 1e-6), 2)
    iou = np.round(class_wise_metrics["TP"] / (class_wise_metrics["TP"] + class_wise_metrics["FP"] + class_wise_metrics["FN"] + 1e-6), 2)

    # overall metrics
    overall_recall = np.mean(recall)
    overall_precision = np.mean(precision)
    overall_iou = np.mean(iou)

    # package evaluations
    evaluations = {
        "class_wise": {
            "Recall": recall.tolist(),
            "Precision": precision.tolist(),
            "IoU": iou.tolist()
        },
        "overall": {
            "Recall": round(overall_recall, 2),
            "Precision": round(overall_precision, 2),
            "IoU": round(overall_iou, 2)
        }
    }

    return evaluations


def show_evaluations(evaluations):
    class_wise_data = evaluations['class_wise']
    class_wise_df = pd.DataFrame(class_wise_data)
    class_wise_df.index = [f"Class {i+1}" for i in range(len(class_wise_data['Recall']))]
    
    overall_data = evaluations['overall']
    overall_df = pd.DataFrame(overall_data, index=['All Classes'])
    
    # Combine 
    combined_df = pd.concat([overall_df, class_wise_df]).reset_index()
    combined_df.rename(columns={'index': 'Class'}, inplace=True)
    
    return combined_df

***fcn***

In [ ]:
evaluations = evaluate_metrics(train_dataset, fcn_model1)
show_evaluations(evaluations)

In [ ]:
evaluations = evaluate_metrics(val_dataset, fcn_model1)
show_evaluations(evaluations)

In [ ]:
evaluations = evaluate_metrics(test_dataset, fcn_model1)
show_evaluations(evaluations)

***unet***

In [ ]:
evaluations = evaluate_metrics(train_dataset, unet_model1)
show_evaluations(evaluations)

In [ ]:
evaluations = evaluate_metrics(val_dataset, unet_model1)
show_evaluations(evaluations)

In [ ]:
evaluations = evaluate_metrics(test_dataset, unet_model1)
show_evaluations(evaluations)

***deeplab***

In [ ]:
evaluations = evaluate_metrics(train_dataset, deeplab_model1)
show_evaluations(evaluations)

In [ ]:
evaluations = evaluate_metrics(val_dataset, deeplab_model1)
show_evaluations(evaluations)

In [ ]:
evaluations = evaluate_metrics(test_dataset, deeplab_model1)
show_evaluations(evaluations)

# 5. Prediction

* ### Predict on Train, Val, Test Dataset

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
def show_predictions(dataset, num, model):
    """
    Displays the first image of each of the num batches
    """
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
# fcn
show_predictions(train_dataset, 3, fcn_model1)
show_predictions(val_dataset, 3, fcn_model1)
show_predictions(test_dataset, 3, fcn_model1)

In [ ]:
# unet
show_predictions(train_dataset, 3, unet_model1)
show_predictions(val_dataset, 3, unet_model1)
show_predictions(test_dataset, 3, unet_model1)

In [ ]:
# deeplab
show_predictions(train_dataset, 3, deeplab_model1)
show_predictions(val_dataset, 3, deeplab_model1)
show_predictions(test_dataset, 3, deeplab_model1)

* ### Predict on New Dataset

In [ ]:
from pathlib import Path

def load_and_preprocess_image(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [256, 256], method='nearest')  
    image = tf.image.convert_image_dtype(image, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.image.resize(mask, [256, 256], method='nearest')  
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)

    return image, mask

# Paths to the new dataset
IMAGE_PATH = Path("/kaggle/input/semantic-segmentation-car-driving/x-rgb")
MASK_PATH = Path("/kaggle/input/semantic-segmentation-car-driving/y-mask")
new_images_paths = sorted([str(x) for x in IMAGE_PATH.glob("*.png")])
new_masks_paths = sorted([str(x) for x in MASK_PATH.glob("*.png")])

# Creating the new dataset
new_dataset = tf.data.Dataset.from_tensor_slices((new_images_paths, new_masks_paths))
new_dataset = new_dataset.map(load_and_preprocess_image)
new_dataset_batched = new_dataset.batch(1)

In [ ]:
# fcn
show_predictions(new_dataset_batched, 5, fcn_model1)

In [ ]:
# unet
show_predictions(new_dataset_batched, 5, unet_model1)

In [ ]:
#deeplab
show_predictions(new_dataset_batched, 5, deeplab_model1)

# 6. Save Model

In [ ]:
from tensorflow.keras.models import load_model
fcn_model1.save('fcn1-image-segmentation-model.h5')

unet_model1.save('unet1-image-segmentation-model.h5')

deeplab_model1.save('deeplab1-image-segmentation-model.h5')